In [19]:
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.tree import DecisionTreeClassifier

In [20]:
df = pd.read_csv('train_prepared.csv')

In [21]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,0,1,0,0,0,1,1,66,39,0,0,0,0,0,0,0,80,0
1,1,0,0,0,0,1,0,70,24,0,109,9,25,549,44,0,83,1
2,0,1,0,0,0,1,0,65,58,1,43,3576,0,6715,49,0,83,0
3,0,1,0,0,0,1,0,65,33,0,0,1283,371,3329,193,0,83,0
4,1,0,0,0,0,1,0,70,16,0,303,70,151,565,2,1,83,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,0,1,0,1,0,0,0,65,41,1,0,6819,0,1643,74,98,80,0
8689,1,0,0,0,1,0,1,71,18,0,0,0,0,0,0,1499,83,0
8690,1,0,0,0,0,1,1,71,26,0,0,0,0,0,0,1500,83,1
8691,0,1,0,1,0,0,0,69,32,0,0,1049,0,353,3235,608,83,0


In [22]:
y = df['17']
x = df[['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16']]

In [23]:
x_train, x_test_val, y_train, y_test_val = train_test_split(x, y, test_size=0.2)

In [24]:
random_forest = RandomForestClassifier(criterion='gini' , n_estimators=360, random_state=50)
#random_forest = DecisionTreeClassifier(criterion='entropy', random_state=42)
xgb_model = xgb.XGBClassifier()
ensemble_model = xgb.XGBClassifier(base_estimator=random_forest, n_estimators=10)

In [25]:
ensemble_model.fit(x_train, y_train)

[08:47:49] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "base_estimator" } are not used.



XGBClassifier(base_estimator=RandomForestClassifier(n_estimators=360,
                                                    random_state=50),
              base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=10, n_jobs=None, num_parallel_tree=None,
              predictor=None, ...)

In [26]:
y_pred = ensemble_model.predict(x_test_val)

In [27]:
accuracy = accuracy_score(y_test_val, y_pred)
print("Độ chính xác:", accuracy)

Độ chính xác: 0.7659574468085106


In [28]:
df = pd.read_csv('test.csv')
df = df[['PassengerId', 'HomePlanet']]
df

,PassengerId,HomePlanet
0,0013_01,Earth
1,0018_01,Earth
2,0019_01,Europa
3,0021_01,Europa
4,0023_01,Earth
...,...,...
4272,9266_02,Earth
4273,9269_01,Earth
4274,9271_01,Mars
4275,9273_01,Europa


In [29]:
x_sub = pd.read_csv('test_prepared.csv')
x_sub
y_sub = ensemble_model.predict(x_sub)
len(y_sub)

4277

In [30]:
for row in range(len(df)):
    df.iloc[row, 1] = y_sub[row]
    if df.iloc[row, 1] == 0:
        df.iloc[row, 1] = False
    else:
        df.iloc[row, 1] = True
df = df.rename(columns={'HomePlanet': 'Transported'})
df

,PassengerId,Transported
0,0013_01,False
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,False
...,...,...
4272,9266_02,True
4273,9269_01,False
4274,9271_01,True
4275,9273_01,True


In [31]:
df.to_csv('submit.csv', index=False)